## Clone out InterFaceGAN



In [ ]:
!git clone https://github.com/genforce/interfacegan.git InterFaceGAN && cd InterFaceGAN && git checkout b707e942187f464251f855c92f7009b8cf13bf03

In [1]:
import sys
import logging
import numpy as np
import cv2
from glob import glob
from pathlib import Path
from collections import defaultdict
from tqdm.notebook import tqdm

from InterFaceGAN.models.stylegan_generator import StyleGANGenerator

In [2]:
# set up logging
logger = logging.getLogger('generate_data')

if not logger.handlers:
    sh = logging.StreamHandler(sys.stdout)
    sh.setFormatter(logging.Formatter(
        fmt='%(asctime)s [%(levelname)s] %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
    ))
    logger.addHandler(sh)

In [3]:
# Colab
# from google.colab import drive
# drive.mount('/content/drive')

## Download the Pretrained Model

In [4]:
MODEL_NAME = 'stylegan_ffhq'
!wget https://www.dropbox.com/s/qyv37eaobnow7fu/{MODEL_NAME}.pth?dl=1 -O InterFaceGAN/models/pretrain/{MODEL_NAME}.pth --quiet    
model = StyleGANGenerator(MODEL_NAME)

[2022-11-20 00:20:25,258][INFO] Loading pytorch model from `InterFaceGAN/models/pretrain/stylegan_ffhq.pth`.
[2022-11-20 00:20:25,397][INFO] Successfully loaded!
[2022-11-20 00:20:25,399][INFO]   `lod` of the loaded model is 0.0.


## Generate Images

To Each run generate only 10k images to fit into Kaggle limits. It takes around 15 minutes and the final zip file has around 3 GB.

In [8]:
RUN_NUM = 0
NUM_IMAGES = 10

TMP_DIR = Path('tmp')/MODEL_NAME
!rm -r {TMP_DIR}
!mkdir -p {TMP_DIR}
# !python InterFaceGAN/generate_data.py -m {MODEL_NAME} -o {TMP_DIR} -n {NUM_IMAGES}

logger.info(f'Started run #{RUN_NUM}')
logger.info(f'Started generating {NUM_IMAGES} images.')

latent_codes = model.easy_sample(NUM_IMAGES)
results = defaultdict(list)
pbar = tqdm(total=NUM_IMAGES, leave=False)


for latent_codes_batch in model.get_batch_inputs(latent_codes):
    outputs = model.easy_synthesize(latent_codes_batch)
    for key, val in outputs.items():
        if key == 'image':
            for image in val:
                cv2.imwrite(str(TMP_DIR/f'{pbar.n + RUN_NUM * NUM_IMAGES:06d}.jpg'), image[:, :, ::-1])
                pbar.update(1)
        else:
            results[key].append(val)
    if 'image' not in outputs:
        pbar.update(latent_codes_batch.shape[0])
pbar.close()

logger.info(f'Finished generating {NUM_IMAGES} images.')

for key, val in results.items():
    result_path = TMP_DIR/f'{key}_{RUN_NUM:02d}.npy'
    np.save(result_path, np.concatenate(val, axis=0))
    logger.info(f'Saved result {key} to {result_path}')

logger.info(f'Finished run #{RUN_NUM}')

rm: cannot remove 'tmp/stylegan_ffhq': No such file or directory


  0%|          | 0/10 [00:00<?, ?it/s]

## Zip the Images and dlatents

In [ ]:
ZIP_PATH = Path('data')/f'{MODEL_NAME}_{RUN_NUM:02d}.zip'
logger.info(f'Started compressing data to {ZIP_PATH}')
!zip -jqr {ZIP_PATH} {TMP_DIR}
logger.info(f'Finished compressing data to {ZIP_PATH}')

## Combine Data

After unziping all the folders and moving all the files to `data/stylegan_ffhq` folder run the following cell to concatenate the dlatents

In [6]:
DATA_DIR = Path('data')/MODEL_NAME

In [19]:
for latent_space_type in ['w', 'wp', 'z']:
    paths = sorted(glob(str(DATA_DIR/f'{latent_space_type}_*.npy')))
    dlatents = np.vstack(list(map(np.load, paths)))
    np.save(DATA_DIR/f'{latent_space_type}.npy', dlatents)